In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/03-training-experiment-tracking/mlruns/2', creation_time=1725119243924, experiment_id='2', last_update_time=1725119243924, lifecycle_stage='active', name='my-cool_experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/03-training-experiment-tracking/mlruns/1', creation_time=1725103907898, experiment_id='1', last_update_time=1725103907898, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1725103092179, experiment_id='0', last_update_time=1725103092179, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
client.create_experiment(name = "my-cool_experiment")

MlflowException: Experiment(name=my-cool_experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-cool_experiment', None, 'active', 1725171614605, 1725171614605)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [5]:
from mlflow.entities import ViewType

## Find best models/runs in given experiment

In [6]:
runs = client.search_runs(
    experiment_ids= '1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [7]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 5dd524e4804c4dd7a86a1874efd0efeb, rmse: 6.4798


# Promote model to model registry

In [8]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [9]:
run_id = "5dd524e4804c4dd7a86a1874efd0efeb"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="NYC trip Duration")

Registered model 'NYC trip Duration' already exists. Creating a new version of this model...
Created version '4' of model 'NYC trip Duration'.


<ModelVersion: aliases=[], creation_timestamp=1725171639733, current_stage='None', description=None, last_updated_timestamp=1725171639733, name='NYC trip Duration', run_id='5dd524e4804c4dd7a86a1874efd0efeb', run_link=None, source='/workspaces/mlops-zoomcamp/03-training-experiment-tracking/mlruns/1/5dd524e4804c4dd7a86a1874efd0efeb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [10]:
model_name = "NYC trip Duration"
latest_versions = client.get_latest_versions(name=model_name)

/tmp/ipykernel_13388/2772786852.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [11]:
latest_versions

[<ModelVersion: aliases=[], creation_timestamp=1725171639733, current_stage='None', description=None, last_updated_timestamp=1725171639733, name='NYC trip Duration', run_id='5dd524e4804c4dd7a86a1874efd0efeb', run_link=None, source='/workspaces/mlops-zoomcamp/03-training-experiment-tracking/mlruns/1/5dd524e4804c4dd7a86a1874efd0efeb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>,
 <ModelVersion: aliases=[], creation_timestamp=1725120156043, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-08-31 16:14:28.260655', last_updated_timestamp=1725120868261, name='NYC trip Duration', run_id='5dd524e4804c4dd7a86a1874efd0efeb', run_link=None, source='/workspaces/mlops-zoomcamp/03-training-experiment-tracking/mlruns/1/5dd524e4804c4dd7a86a1874efd0efeb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>]

In [12]:
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 4, stage: None
version: 3, stage: Staging


## Change Stage of model

In [13]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_13388/1957332551.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1725120156043, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-08-31 16:14:28.260655', last_updated_timestamp=1725171671243, name='NYC trip Duration', run_id='5dd524e4804c4dd7a86a1874efd0efeb', run_link=None, source='/workspaces/mlops-zoomcamp/03-training-experiment-tracking/mlruns/1/5dd524e4804c4dd7a86a1874efd0efeb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [14]:
from datetime import datetime

date = datetime.today()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1725120156043, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-09-01 06:21:24.780378', last_updated_timestamp=1725171684781, name='NYC trip Duration', run_id='5dd524e4804c4dd7a86a1874efd0efeb', run_link=None, source='/workspaces/mlops-zoomcamp/03-training-experiment-tracking/mlruns/1/5dd524e4804c4dd7a86a1874efd0efeb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

# Promote model to production

In [31]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
    else:    
        df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model("runs:/5dd524e4804c4dd7a86a1874efd0efeb/models_mlflow")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [16]:
df = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet")

In [17]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/workspaces/mlops-zoomcamp/03-training-experiment-tracking/preprocessor'

In [18]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [19]:
X_test = preprocess(df, dv)

In [21]:
target ='duration'
y_test = df[target].values

In [23]:
model_name

'NYC trip Duration'

In [32]:
%time test_model(name = "NYC trip Duration",stage= '3', X_test=X_test, y_test=y_test)

CPU times: user 2.95 s, sys: 237 ms, total: 3.19 s
Wall time: 3.01 s


/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': np.float64(6.414347681138584)}

In [33]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_13388/684772386.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1725120156043, current_stage='Production', description='The model version 3 was transitioned to Staging on 2024-09-01 06:21:24.780378', last_updated_timestamp=1725172909808, name='NYC trip Duration', run_id='5dd524e4804c4dd7a86a1874efd0efeb', run_link=None, source='/workspaces/mlops-zoomcamp/03-training-experiment-tracking/mlruns/1/5dd524e4804c4dd7a86a1874efd0efeb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>